## Подготовка к работе с Википедией

Скачайте 'enwiki-20200401-pages-articles.xml.bz2' по ссылке https://meta.wikimedia.org/wiki/Data_dump_torrents — архив весит порядка 16Гб

Скачайте 'wiki.corpus' по ссылке https://yadi.sk/d/TVo-KPUbgx4vPA — это слепок памяти объекта для работы с нелемматизированной(!) Википедией

In [1]:
import os.path
from gensim.corpora.wikicorpus import WikiCorpus

In [2]:
if os.path.isfile("wiki.corpus"):
    print("Using preloaded object")
    wiki = WikiCorpus.load("wiki.corpus")
else:
    print("Now wait for about 15 hours...")
    import logging
    logging.basicConfig(format='%(asctime)s: %(levelname)s: %(message)s')
    logging.root.setLevel(level=logging.INFO)
    wiki = WikiCorpus('enwiki-20200401-pages-articles.xml.bz2', lemmatize=False)
    wiki.save('wiki.corpus')

Using preloaded object


In [ ]:
for text in wiki.get_texts():
    print(text, type(text))
    break    

## Сначала рассчитаем корреляцию Спирмена с Жаккаром по окну радиуса 2

In [4]:
from scipy.stats import spearmanr

In [14]:
f=open("SimLex-999.txt", 'r').readlines()

simlex_words=set()
for i in f[1:]:
    ii=i.split('\t')
    simlex_words.add(ii[0])
    simlex_words.add(ii[1])

pseudodocs={i:set() for i in simlex_words}

counter=0
k=2#радиус окна

for text in wiki.get_texts():
    counter+=1
  
    for ind, word in enumerate(text):
        if word in simlex_words:
            l = ind - k if ind - k > 0 else 0
            r = ind + k + 1
            pseudodocs[word].update(text[l:r])
            
    if counter % 10000 == 0:
        simlex_pairs=[]
        jaccard_pairs=[]
        for i in f[1:]:
            ii=i.split('\t')
            w1=ii[0]
            w2=ii[1]
            try:
                jaccard_pairs.append(\
                                    len(pseudodocs[w1].intersection(pseudodocs[w2]))*1./ \
                                    len(pseudodocs[w1].union(pseudodocs[w2]))
                                    )
                simlex_pairs.append(float(ii[3]))
            except:
                print(pseudodocs[w1], pseudodocs[w2])

        coef = spearmanr(simlex_pairs, jaccard_pairs)
        print(counter, coef)
    
    if counter == 200000:
        break

10000 SpearmanrResult(correlation=0.11064909248070758, pvalue=0.0004588995563067517)
20000 SpearmanrResult(correlation=0.1084351638660099, pvalue=0.0005966280197246219)
30000 SpearmanrResult(correlation=0.10610753479225185, pvalue=0.0007821809303677304)
40000 SpearmanrResult(correlation=0.10009823418856723, pvalue=0.0015359764071859177)
50000 SpearmanrResult(correlation=0.09605745331446396, pvalue=0.002371178665128621)
60000 SpearmanrResult(correlation=0.09712560476989328, pvalue=0.0021172657885964903)
70000 SpearmanrResult(correlation=0.09752549212977182, pvalue=0.0020287959415915898)
80000 SpearmanrResult(correlation=0.09722981797062427, pvalue=0.0020938758309202955)
90000 SpearmanrResult(correlation=0.09331948912684855, pvalue=0.003154191743651448)
100000 SpearmanrResult(correlation=0.08980522971504232, pvalue=0.00450205362115943)
110000 SpearmanrResult(correlation=0.08784360372103074, pvalue=0.005463309353402319)
120000 SpearmanrResult(correlation=0.08511109500618216, pvalue=0.0071

После обработки первых 200000 статей можно остановиться и сделать вывод, что корреляция вряд ли станет больше 0.12

Но что интересно, и меньше 0.08 она тоже не стала, что означает, что числитель и знаменатель в коэффициенте Жаккара растут достаточно равномерно

Поэтому имеет смысл попробовать окна другого радиуса

## Затем произведём одновременный расчёт корреляции Спирмена с Жаккаром по нескольким окнам

In [4]:
from scipy.stats import spearmanr

In [5]:
f=open("SimLex-999.txt", 'r').readlines()

simlex_words=set()
for i in f[1:]:
    ii=i.split('\t')
    simlex_words.add(ii[0])
    simlex_words.add(ii[1])

pseudodocs={k:{i:set() for i in simlex_words} for k in [1, 2, 3, 4, 5]}

counter=0

for text in wiki.get_texts():
    counter+=1
    
    for ind, word in enumerate(text):
        if word in simlex_words:
            for k in [1, 2, 3, 4, 5]:
                l = ind - k if ind - k > 0 else 0
                r = ind + k + 1
                pseudodocs[k][word].update(text[l:r])
                
    if counter % 10000 == 0:
        for k in [1, 2, 3, 4, 5]:
            simlex_pairs=[]
            jaccard_pairs=[]
            for i in f[1:]:
                ii=i.split('\t')
                first_word=ii[0]
                second_word=ii[1]
                try:
                    jaccard_pairs.append(\
                                        len(pseudodocs[k][first_word].intersection(pseudodocs[k][second_word]))*1./ \
                                        len(pseudodocs[k][first_word].union(pseudodocs[k][second_word]))
                                        )
                    simlex_pairs.append(float(ii[3]))
                except:
                    print(k, pseudodocs[k][first_word], pseudodocs[k][second_word])

            coef = spearmanr(simlex_pairs, jaccard_pairs)
            print(k, counter, coef)
            
    if counter == 200000:
        break

1 10000 SpearmanrResult(correlation=0.16485923876088465, pvalue=1.6051592219945796e-07)
2 10000 SpearmanrResult(correlation=0.11076630186537834, pvalue=0.0004525067637921213)
3 10000 SpearmanrResult(correlation=0.08899130575289914, pvalue=0.004880613957089424)
4 10000 SpearmanrResult(correlation=0.07452602070841902, pvalue=0.018479444921519598)
5 10000 SpearmanrResult(correlation=0.06691854488336273, pvalue=0.03444574341758788)
1 20000 SpearmanrResult(correlation=0.16696755787891576, pvalue=1.1081451599619917e-07)
2 20000 SpearmanrResult(correlation=0.10843447177938247, pvalue=0.0005966765274165116)
3 20000 SpearmanrResult(correlation=0.08702283110789909, pvalue=0.005917711626231574)
4 20000 SpearmanrResult(correlation=0.07570368187059003, pvalue=0.016702015884979885)
5 20000 SpearmanrResult(correlation=0.06885707952098888, pvalue=0.029538693081851194)
1 30000 SpearmanrResult(correlation=0.16286050964575358, pvalue=2.2709115871863328e-07)
2 30000 SpearmanrResult(correlation=0.106107534

1 200000 SpearmanrResult(correlation=0.11080410684966215, pvalue=0.0004504625820866351)
2 200000 SpearmanrResult(correlation=0.08467571233738665, pvalue=0.007410708344663224)
3 200000 SpearmanrResult(correlation=0.07524560477862735, pvalue=0.017374742770183336)
4 200000 SpearmanrResult(correlation=0.0715996442910879, pvalue=0.023628355981486077)
5 200000 SpearmanrResult(correlation=0.07045699443926792, pvalue=0.025953196950864087)


После обработки первых 200000 статей можно сделать вывод, что наилучший результат показало окно с радиусом 1

## Наконец произведём раздельный расчёт по частям речи по нескольким окнам

In [6]:
from scipy.stats import spearmanr

In [7]:
f=open("SimLex-999.txt", 'r').readlines()

simlex_words={w: set() for w in ['A', 'N', 'V']}
for i in f[1:]:
    ii=i.split('\t')
    simlex_words[ii[2]].add(ii[0])
    simlex_words[ii[2]].add(ii[1])

pseudodocs={k:{w:{i:set() for i in simlex_words[w]} for w in ['A', 'N', 'V']} for k in [1, 2, 3, 4, 5]}

counter=0

for text in wiki.get_texts():
    counter+=1

    for ind, word in enumerate(text):
        ww=None
        if word in simlex_words['A']:
            ww='A'
        elif word in simlex_words['N']:
            ww='N'
        elif word in simlex_words['V']:
            ww='V'
        if ww:
            for k in [1, 2, 3, 4, 5]:
                l = ind - k if ind - k > 0 else 0
                r = ind + k + 1
                pseudodocs[k][ww][word].update(text[l:r])

    if counter % 10000 == 0:
        for k in [1, 2, 3, 4, 5]:
            for w in ['A', 'N', 'V']:
                simlex_pairs=[]
                jaccard_pairs=[]
                for i in f[1:]:
                    ii=i.split('\t')
                    first_word=ii[0]
                    second_word=ii[1]
                    try:
                        if ii[2] == w:
                            jaccard_pairs.append(\
                                                len(pseudodocs[k][w][first_word].intersection(pseudodocs[k][w][second_word]))*1./ \
                                                len(pseudodocs[k][w][first_word].union(pseudodocs[k][w][second_word]))
                            )
                            simlex_pairs.append(float(ii[3]))

                    except:
                        print(k, w, first_word, second_word)#, pseudodocs[k][w][first_word], pseudodocs[k][w][second_word])

                coef = spearmanr(simlex_pairs, jaccard_pairs)
                print(k, w, counter, coef)
            
    if counter == 200000:
        break

1 A 10000 SpearmanrResult(correlation=0.17291892033620188, pvalue=0.06954110078372773)
1 N 10000 SpearmanrResult(correlation=0.18481679083251246, pvalue=1.57064978131956e-06)
1 V 10000 SpearmanrResult(correlation=0.09450634729802541, pvalue=0.16052547565075057)
2 A 10000 SpearmanrResult(correlation=0.13670448986759207, pvalue=0.152518135611169)
2 N 10000 SpearmanrResult(correlation=0.12362766199507362, pvalue=0.001389931291825453)
2 V 10000 SpearmanrResult(correlation=0.07415337141694049, pvalue=0.27127180418231567)
3 A 10000 SpearmanrResult(correlation=0.11982650652037191, pvalue=0.21032272798104482)
3 N 10000 SpearmanrResult(correlation=0.10164166383149141, pvalue=0.008666487247528852)
3 V 10000 SpearmanrResult(correlation=0.05892213193284733, pvalue=0.3822689610629747)
4 A 10000 SpearmanrResult(correlation=0.09806739246147103, pvalue=0.3058452580187223)
4 N 10000 SpearmanrResult(correlation=0.09188867804800506, pvalue=0.01769524336270219)
4 V 10000 SpearmanrResult(correlation=0.0427

2 V 70000 SpearmanrResult(correlation=0.07093963637294864, pvalue=0.29264530659766386)
3 A 70000 SpearmanrResult(correlation=0.1047412405122047, pvalue=0.2739382977003108)
3 N 70000 SpearmanrResult(correlation=0.10155958526248934, pvalue=0.008720872023690861)
3 V 70000 SpearmanrResult(correlation=0.050913021594544076, pvalue=0.4503724820770486)
4 A 70000 SpearmanrResult(correlation=0.09116537866541313, pvalue=0.34130111876117275)
4 N 70000 SpearmanrResult(correlation=0.09418949442441157, pvalue=0.015032812550175753)
4 V 70000 SpearmanrResult(correlation=0.03811731066512119, pvalue=0.5721184180585231)
5 A 70000 SpearmanrResult(correlation=0.08958576965932233, pvalue=0.34976659460157034)
5 N 70000 SpearmanrResult(correlation=0.09036606709727789, pvalue=0.019676092679479576)
5 V 70000 SpearmanrResult(correlation=0.0332067673914176, pvalue=0.6226399341584059)
1 A 80000 SpearmanrResult(correlation=0.17770162538242124, pvalue=0.06205692456356705)
1 N 80000 SpearmanrResult(correlation=0.16613

4 A 130000 SpearmanrResult(correlation=0.07899361371292452, pvalue=0.4098730038111129)
4 N 130000 SpearmanrResult(correlation=0.09407445037223941, pvalue=0.015157061176552574)
4 V 130000 SpearmanrResult(correlation=0.02754269648111931, pvalue=0.6831725900114339)
5 A 130000 SpearmanrResult(correlation=0.07712002191958903, pvalue=0.4211017296089018)
5 N 130000 SpearmanrResult(correlation=0.0902052410370556, pvalue=0.01989619585854566)
5 V 130000 SpearmanrResult(correlation=0.022405656181277715, pvalue=0.7398939076077697)
1 A 140000 SpearmanrResult(correlation=0.1280668001688123, pvalue=0.1804025241452368)
1 N 140000 SpearmanrResult(correlation=0.14091522464609002, pvalue=0.00026430780392394947)
1 V 140000 SpearmanrResult(correlation=0.08112706677861292, pvalue=0.22861750227785385)
2 A 140000 SpearmanrResult(correlation=0.08588685190339301, pvalue=0.37009867331104085)
2 N 140000 SpearmanrResult(correlation=0.10619110362346286, pvalue=0.006086530665786748)
2 V 140000 SpearmanrResult(correl

5 N 190000 SpearmanrResult(correlation=0.09593071377760887, pvalue=0.01325864744112122)
5 V 190000 SpearmanrResult(correlation=0.021364208596202538, pvalue=0.7515784966021003)
1 A 200000 SpearmanrResult(correlation=0.12738230293283961, pvalue=0.1827622536708083)
1 N 200000 SpearmanrResult(correlation=0.1381978660473362, pvalue=0.0003476314701701081)
1 V 200000 SpearmanrResult(correlation=0.07266715606383162, pvalue=0.28102042255295634)
2 A 200000 SpearmanrResult(correlation=0.09299509243080165, pvalue=0.3316584406759212)
2 N 200000 SpearmanrResult(correlation=0.10999123371757369, pvalue=0.004485460087239874)
2 V 200000 SpearmanrResult(correlation=0.05322460746663037, pvalue=0.4300468680720049)
3 A 200000 SpearmanrResult(correlation=0.07862503827817001, pvalue=0.4120679534223133)
3 N 200000 SpearmanrResult(correlation=0.10330242953167779, pvalue=0.00762914249506186)
3 V 200000 SpearmanrResult(correlation=0.03773506265732899, pvalue=0.5759808673252795)
4 A 200000 SpearmanrResult(correlat

После обработки первых 200000 статей можно сделать вывод, что лучший результат показали существительные

## To Do

Проверить, что направления шкал совпадают — т.е. больше значит ближе

Провести расчёты для лемматизированной версии Википедии

Выяснить, как организовать доступ за O(1) к произвольной статье